In [207]:
import spacy
import nltk
import pandas as pd

from spacy import displacy
from spacy.matcher import Matcher

In [269]:
nlp = spacy.load('de')
#text = u'''Meine kleine Enkelin Lisa und mein Enkel Lukas fliegen morgen nach London.'''
#text = u'''Herbert ist der Vater von Hans'''
text = u'''Peter und Maria gehen morgen ins Kino'''

In [270]:
text_long = u'''Meine kleine Enkelin Lisa und mein Enkel Lukas fliegen morgen nach London. Sie sind zum ersten Mal in England. 
        Peter und Maria gehen morgen ins Kino. Ich und mein Sohn gehen heute zum Fußball. 
        Ich gehe mit Johann in den Zoo. Hans und sein Sohn Hubert.'''

In [271]:
sentences = nltk.sent_tokenize(text)
sentences

['Peter und Maria gehen morgen ins Kino']

In [272]:
import pandas as pd
feature_columns = ['ne', 'ne_dep', 'ne_head', 'ne_pos', 'lemma', 'children']
features = pd.DataFrame(columns=feature_columns)

In [273]:
for i, sentence in enumerate(sentences):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        ne = token.text
        ne_dep = token.dep_
        head = token.head.text
        pos = token.head.pos_
        children = [child for child in token.children]
        lemma = token.lemma_
        #print(token.text, token.dep_, token.head.text, token.head.pos_,
         # [child for child in token.children])
        
        data = {'ne': ne.lower(), 'ne_dep': ne_dep.lower(), 'ne_head': head.lower(), 
                'ne_pos': pos.lower(), 'lemma': lemma.lower(),'children': children}
        training_ex = pd.Series(data, index=feature_columns)
        features = features.append(training_ex, ignore_index=True)
    
    displacy.render(doc, style='dep', jupyter=True)
features
    

,ne,ne_dep,ne_head,ne_pos,lemma,children
0,peter,sb,gehen,verb,peter,[und]
1,und,cd,peter,propn,und,[Maria]
2,maria,cj,und,conj,maria,[]
3,gehen,root,gehen,verb,gehen,"[Peter, morgen, ins]"
4,morgen,mo,gehen,verb,morgen,[]
5,ins,mo,gehen,verb,ins,[Kino]
6,kino,nk,ins,adp,kino,[]


In [200]:
entities = ['vater', 'mutter', 'sohn', 'tochter', 'bruder', 'schwester', 'enkel', 'enkelin', 'nichte',
            'neffe', 'onkel', 'tante', 'ich', 'meine', 'mein', 'meinen', 'meines', 'lisa', 'max', 'hans', 'hubert', 'lukas',
           'london', 'herbert', 'peter', 'maria', 'zoo']

TypeError: Argument 'other' has incorrect type (expected spacy.tokens.token.Token, got str)

In [236]:
rel_ents = features.loc[features['ne'].isin(entities)]
for ent in rel_ents.iterrows():
    print(ent[1]['ne'])

meine
enkelin
lisa
mein
enkel
lukas
london


In [201]:
for i, row in enumerate(features['ne'].iteritems()):
    elem1 = row[1].lower()

    if elem1 in entities:
        head = features['ne_head'][i].lower()
        rels = features.loc[(features['ne_head'] == head) & features['ne'].isin(entities) & ~features['ne'].isin([elem1])]

        for rel in rels['ne'].iteritems():
            print(f"{elem1}->{head}->{rel[1]}")

In [65]:
rels = {}

for i, row in enumerate(features['ne'].iteritems()):
    elem1 = row[1].lower()
    if elem1 in entities:
        elem2 = features['ne_head'][i]
        for i, row2 in 

Enkelin
fliegen
Enkelin
Enkel
und
Enkel
ist
von


In [153]:
rels = {}

for i, row in enumerate(features['ne'].iteritems()):
    elem1 = row[1].lower()
    if elem1 in entities:
        head = features['ne_head'][i].lower()
        
        rels = features.loc[(features['ne_head'] == head) & features['ne'].isin(entities)]
        print(rels)
        



      ne ne_dep  ne_head ne_pos  lemma children
0  meine     nk  enkelin   noun  meine       []
3   lisa     nk  enkelin   noun   lisa    [und]
        ne ne_dep  ne_head ne_pos    lemma               children
2  enkelin     sb  fliegen   verb  enkelin  [Meine, kleine, Lisa]
      ne ne_dep  ne_head ne_pos  lemma children
0  meine     nk  enkelin   noun  meine       []
3   lisa     nk  enkelin   noun   lisa    [und]
     ne ne_dep ne_head ne_pos   lemma children
5  mein     nk   enkel   noun  meinen       []
      ne ne_dep ne_head ne_pos  lemma       children
6  enkel     cj     und   conj  enkel  [mein, Lukas]


In [8]:
def iterate(text):
    for token in text:
        children = [child for child in token.children]
        if children:        
            iterate(children)
        
    print(token.text)
        

for token in doc:
    if token.dep_ == 'ROOT':
        children = [child for child in token.children]

        iterate(children)

iterate(doc)
        

Max
von
.
Max
von
.
Max
von
Max
.


In [58]:
for sentence in sentences:
    doc = nlp(sentence)
    
    
    for token in doc:        
        if token.dep_ == 'ROOT':
            for child in token.children:
                print(child)
    for token in doc:
        if token.text = 
            

Herbert
Vater


In [9]:
root = [token for token in doc if token.head == token][0]
subject = list(root.lefts)[0]
for descendant in subject.subtree:
    assert subject is descendant or subject.is_ancestor(descendant)
    print(descendant.text, descendant.dep_, descendant.n_lefts,
          descendant.n_rights,
          [ancestor.text for ancestor in descendant.ancestors])

Herbert pd 0 0 ['ist']
